In [1]:
from load_data import *
from benchmark_1N import * 

/usr/local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Run With FaceNet as Model

In [2]:
# load 10% of the dataset
gallery_data, proxy_data = load_CelebA(0.1, 0.1, 0.1, max_per_id=10)

# Split into first 5 images for actual gallery, rest of images to be used in attack as members
gallery_data_otherimages = gallery_data.copy()
gallery_data['filename'] = gallery_data['filename'].apply(lambda x: x[:5])
gallery_data_otherimages['filename'] = gallery_data_otherimages['filename'].apply(lambda x: x[5:] if len(x) > 5 else [])
gallery_data_otherimages = gallery_data_otherimages[gallery_data_otherimages['filename'].apply(lambda x: len(x) > 0)].reset_index()

# Load 15% of the dataset to be used in the attack as non-members
gallery_data_nonmembers, _ = load_CelebA(0.1, 0.1, 0.15)

Files already downloaded and verified
=========CelebA dataset=========
Number of subjects: 1017
Gallery set # subjects: 921, # images: 8300
Proxy set # subjects: 915, # images: 2661
# of subjects in proxy only: 96
Files already downloaded and verified
=========CelebA dataset=========
Number of subjects: 1526
Gallery set # subjects: 1379, # images: 6551
Proxy set # subjects: 1373, # images: 3870
# of subjects in proxy only: 147


In [3]:
version = 'v0'

In [4]:
model = InceptionResnetV1(pretrained='vggface2').eval()
benchmark = Benchmark(gallery_data, proxy_data)
# run benchmark for 1:N search without adversarial attack to generate templates once
benchmark.run_benchmark(model, SearchTemplate(), new_version_name=version, verbose=True)

/usr/local/lib/python3.9/site-packages/facenet_pytorch/models/inception_resnet_v1.py:329: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(cached_file)


Enrolling gallery templates...
Enrolled gallery templates in 10.49 mins.
Enrolling proxy templates...
Enrolled proxy templates in 8.57 mins.
Searching for proxy images in gallery...
Search completed in 0.04 mins.


In [5]:
print(f'FNR at FPR 0.001 {benchmark.get_fnr_at_fpr_top1(0.001)}')
print(f'FNR at FPR 0.1 {benchmark.get_fnr_at_fpr_top1(0.1)}')

FNR at FPR 0.001 0.8351648351648352
FNR at FPR 0.1 0.4835164835164835


In [ ]:
attack_templates, attack_templates_nonmember = prepare_attack_data(gallery_data_otherimages, gallery_data_nonmembers, model, new_version_name='v0', n_samples=25)

In [21]:
search_attacks1, search_attacks0 = prepare_attack_searches(attack_templates, attack_templates_nonmember, 0.75)

res0 = []
for a in search_attacks0[:10]:
    benchmark.run_benchmark(model, a, load_from_version='v0')
    fnr1 = benchmark.get_fnr_at_fpr_top1(0.1)
    res0.append(fnr1)

res1 = []
for a in search_attacks1[:10]:
    benchmark.run_benchmark(model, a, load_from_version='v0')
    fnr1 = benchmark.get_fnr_at_fpr_top1(0.1)
    res1.append(fnr1)

In [29]:
fpr = np.mean([fnr >= 0.9 for fnr in res0])
tpr = np.mean([fnr >= 0.9 for fnr in res1])
tpr, fpr

(np.float64(1.0), np.float64(0.6))

## Run with ArcNet as Model and Face Detection Preprocessing

In [2]:
# load 5% of the dataset
gallery_data, proxy_data = load_CelebA(0.1, 0.1, 0.05, max_per_id=10)

# Split into first 5 images for actual gallery, rest of images to be used in attack as members
gallery_data_otherimages = gallery_data.copy()
gallery_data['filename'] = gallery_data['filename'].apply(lambda x: x[:5])
gallery_data_otherimages['filename'] = gallery_data_otherimages['filename'].apply(lambda x: x[5:] if len(x) > 5 else [])
gallery_data_otherimages = gallery_data_otherimages[gallery_data_otherimages['filename'].apply(lambda x: len(x) > 0)].reset_index()

# Load 10% of the dataset to be used in the attack as non-members
gallery_data_nonmembers, _ = load_CelebA(0.1, 0.1, 0.10)

Files already downloaded and verified
=========CelebA dataset=========
Number of subjects: 508
Gallery set # subjects: 459, # images: 4122
Proxy set # subjects: 457, # images: 1340
# of subjects in proxy only: 49
Files already downloaded and verified
=========CelebA dataset=========
Number of subjects: 1017
Gallery set # subjects: 921, # images: 4369
Proxy set # subjects: 915, # images: 2661
# of subjects in proxy only: 96


In [3]:
version = 'v0'

In [48]:
# reload benchmark_1N using importlib
import importlib
import benchmark_1N
importlib.reload(benchmark_1N)

<module 'benchmark_1N' from '/workspaces/face_recognition_privacy_attack/benchmark_1N.py'>

In [4]:
model = ArcFaceModel()
benchmark = Benchmark(gallery_data, proxy_data)
# run benchmark for 1:N search without adversarial attack to generate templates once
benchmark.run_benchmark(model, SearchTemplate(), new_version_name=version, verbose=True)

/usr/local/lib/python3.9/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /root/.insightface/models/buffalo_l/genderage.onnx genderage
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)
Enrolling gallery templates...
Enrolled gallery templates in 30.50 mins.
Enrolling proxy templ

In [6]:
print(f'FNR at FPR 0.001 {benchmark.get_fnr_at_fpr_top1(0.001)}')
print(f'FNR at FPR 0.05 {benchmark.get_fnr_at_fpr_top1(0.05)}')

FNR at FPR 0.001 0.051470588235294115
FNR at FPR 0.05 0.049019607843137254


In [7]:
attack_templates, attack_templates_nonmember = prepare_attack_data(gallery_data_otherimages, gallery_data_nonmembers, model, new_version_name='v0', n_samples=25)

In [15]:
search_attacks1, search_attacks0 = prepare_attack_searches(attack_templates, attack_templates_nonmember, 0.8)

res0 = []
for a in search_attacks0[:25]:
    benchmark.run_benchmark(model, a, load_from_version='v0')
    fnr1 = benchmark.get_fnr_at_fpr_top1(0.001)
    res0.append(fnr1)

res1 = []
for a in search_attacks1[:25]:
    benchmark.run_benchmark(model, a, load_from_version='v0')
    fnr1 = benchmark.get_fnr_at_fpr_top1(0.001)
    res1.append(fnr1)

In [19]:
np.mean(res0)

np.float64(0.4688235294117647)

In [18]:
np.mean(res1)

np.float64(0.8102941176470587)

In [20]:
np.mean(res)

NameError: name 'res' is not defined